<a href="https://colab.research.google.com/github/aaron020/CNN_Project/blob/main/resNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Based on tutorial from : https://www.youtube.com/watch?v=glmowUlqoYw

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os 

In [2]:
#pull in a dataset from your computer
pathName = "Shoe vs Sandal vs Boot Dataset"


footwear_types = os.listdir(pathName)

print("The types of footwear are = " , footwear_types)

The types of footwear are =  ['Shoe', 'Sandal', 'Boot']


In [3]:
#Tack all the data from the datset 
footwear = []

for item in footwear_types:
  # get all the files from each folder 
  all_footwear = os.listdir(pathName + '/' + item)

  for fw in all_footwear:
    footwear.append((item, str(pathName + '/' + item) + '/' + fw))

print(footwear[0])

('Shoe', 'Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (4002).jpg')


In [4]:
#Store the folder name in a seperate column - the folder name is what the image is

#Build a dataframe using pandas

footwear_df = pd.DataFrame(data=footwear, columns=['footwear type', 'image'])

print(footwear_df.head())

  footwear type                                              image
0          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (4002...
1          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (388)...
2          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (4868...
3          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (3740...
4          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (1919...


In [5]:
#Print info about the dataset

print('Total number of rooms in the dataset: ', len(footwear))

footwear_count = footwear_df['footwear type'].value_counts()

print('Rooms in each category')
print(footwear_count)

Total number of rooms in the dataset:  15000
Rooms in each category
Shoe      5000
Sandal    5000
Boot      5000
Name: footwear type, dtype: int64


In [6]:
import cv2
path = 'Shoe vs Sandal vs Boot Dataset/'

#Original image size = 136x102, now we resize to 244x244

img_size = 244


images = []
labels = []

for i in footwear_types:
  data_path = path + str(i) #enter into first folder and second and then third 
  filenames = [i for i in os.listdir(data_path)] # name of each image 

  for f in filenames:
    img = cv2.imread(data_path + '/' + f)
    img = cv2.resize(img, (img_size, img_size))
    images.append(img)
    labels.append(i)

In [7]:
#Transform the image array to a numpy type 
images = np.array(images)
images.shape 
#Shape ->  total No. of images x size x dimensions (RGB)

(15000, 244, 244, 3)

In [8]:
images = images.astype(np.float16) / 255.0
#np.float16 or it will run out of ram
#ie divide all the pixel values by 255 just to make the pixel value smaller 
#255 = max pixel value everythng else is gonna be like 0.13 0.43

In [9]:
#The labels are in the form of numbers, the computer does not understand that so we convert 
#them to numbers

from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
y = footwear_df['footwear type'].values
y = y.reshape(-1, 1)

onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(y)
print(Y.shape)
print(Y.toarray())



(15000, 3)
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [10]:
#Divide datset into train and test

from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.10, random_state=415)
#Very small test set as the dataset is very small

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)



(13500, 244, 244, 3)
(13500, 3)
(1500, 244, 244, 3)
(1500, 3)


In [12]:
##Start of building RESNET functions


from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image 
from keras.utils import layer_utils 
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform 
import scipy.misc
from matplotlib.pyplot import imshow


In [34]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X


In [35]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


In [36]:
#Resnet algorithm  - all 50 layers

def ResNet50(input_shape=(244, 244, 3), classes=3):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # Average Pooling
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model



In [37]:
model = ResNet50(input_shape=(224,224,3), classes=3)

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_8 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_9[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d_8[0][0]']       
                                )                                                          

In [ ]:
model.fit(train_x, train_y, epochs = 3, batch_size = 32)

In [ ]:
model.evaluate(test_x, test_y)